In [1]:
# 0. Pacotes
import pandas as pd
from unicodedata import normalize
from bs4 import BeautifulSoup
import urllib.request
import networkx as nx
import matplotlib.pyplot as plt
import operator

In [2]:
# 1. Dados
df_reembolsos = pd.read_csv("/home/gustavo/Downloads/Redes/Datasets/reembolso.csv", delimiter = ';')  # junção de tabelas baixadas em http://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/dados-abertos-cota-parlamentar
df_reembolsos = df_reembolsos[df_reembolsos.txtCNPJCPF.notnull()]  # desprezando casos em que o cnpj da empresa é nulo.
df_reembolsos = df_reembolsos[df_reembolsos["numAno"] > 2015]  # focando nos reembolsos a partir de 2015

/home/gustavo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_receitas = pd.read_csv("/home/gustavo/Downloads/Redes/Datasets/receitas_candidatos_2014_brasil .csv", delimiter=';')  # csv baixado em http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais
df_receitas = df_receitas[df_receitas["Cargo"] == "Deputado Federal"] # somente deputados federais

/home/gustavo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 1.1. Criando dicionário com identificador e nome completo do deputado (será necessário porque a tabela de reembolsos não tem o nome completo, só o apelido)
page = urllib.request.urlopen("http://www.camara.leg.br/SitCamaraWS/Deputados.asmx/ObterDeputados")  # link com os dados básicos dos deputados
soup = BeautifulSoup(page, "lxml")
lista_dados_deputados = soup.find_all("deputado")
def remover_acentos(txt):
    return normalize("NFKD", txt).encode("ASCII", "ignore").decode("ASCII")

deputados_dict = {}
for i in range(0,len(lista_dados_deputados)):
    deputados_dict[lista_dados_deputados[i].idecadastro.string] = [remover_acentos(lista_dados_deputados[i].nome.string), lista_dados_deputados[i].urlfoto.string]

In [5]:
# 1.2. Tratamento dos dados
# 1.2.1. Ajustando nomes dos campos
df_reembolsos.columns = ['nome_parlamentar', 'num_agente_politico', 'carteira_parlamentar', 'mandato', 'uf',
                        'partido', 'legislatura', 'sub_cota', 'descricao', 'especificacao_subcota',
                        'descricao_especificacao', 'fornecedor', 'cnpj_cpf_empresa', 'numero_documento_reebolso', 
                         'tipo_documento', 'data_emissao', 'valor_documento', 'valor_glosa', 'valor_liquido',
                        'mes', 'ano', 'parcelado', 'passageiro', 'trecho', 'lote', 'num_ressarcimento', 
                        'vlr_restituicao', 'deputado_id', 'documento_id']

In [6]:
df_receitas.columns = ["seq_candidato", "uf", "sigla_partido", "num_agente_politico", "cargo", 
                       "nome_agente_politico", "cpf_agente_politico", "num_recibo_eleitoral", "num_documento", 
                       "cpf_cnpj_empresa", "nome_empresa", "cod_setor_economico", "nome_setor_economico", 
                       "data_receita", "valor_receita", "tipo_receita", "especie_recurso", "descricao_receita"]

df_receitas['valor_receita'] = df_receitas['valor_receita'].apply(lambda x: float(x.replace(",", ".")))

In [7]:
# 1.2.2. Criando tabelas agregadas
df_reembolso_resumo = df_reembolsos.groupby(['partido', 'uf', 'num_agente_politico', 'fornecedor', 'cnpj_cpf_empresa'
                                            ]).agg({'valor_liquido':['sum','count']}).reset_index()
df_reembolso_resumo.columns = (['partido', 'uf', 'num_agente_politico', 'fornecedor', 'cnpj_cpf_empresa', 
                                'soma_valor_reembolso', 'qtde_reembolsos'])

In [8]:
df_receitas_resumo = df_receitas.groupby(["sigla_partido", "uf", "nome_agente_politico", "nome_empresa", 
                                          "cpf_cnpj_empresa"]).agg({"valor_receita": ['sum', 'count']}).reset_index()
df_receitas_resumo.columns = ["sigla_partido", "uf", "nome_agente_politico", "nome_empresa", 
                              "cpf_cnpj_empresa", "soma_valor_receitas", "qtde_receitas"]

df_receitas_resumo["nome_completo"] = df_receitas_resumo["nome_agente_politico"].map(remover_acentos)

In [11]:
# 1.2.3. Ajuste do Nome dos deputados (na tabela de reembolsos só tem o apelido)
def nome_completo(x):
    if str(int(x)) in list(deputados_dict.keys()):
        y = deputados_dict[str(int(x))][0]
    else:
        y = "sem_info"
    return y

df_reembolso_resumo["nome_completo"] = df_reembolso_resumo["num_agente_politico"].map(nome_completo)
df_reembolso_resumo["num_agente_politico"] = df_reembolso_resumo.num_agente_politico.apply(str)

In [12]:
# 1.2.4. Ajuste do formato do cnpj/cpf dos geradores do reembolso
df_reembolso_resumo["cnpj_cpf_empresa"] = df_reembolso_resumo["cnpj_cpf_empresa"].apply(
lambda x: x.replace("/",""))
df_reembolso_resumo["cnpj_cpf_empresa"] = df_reembolso_resumo["cnpj_cpf_empresa"].apply(
lambda x: x.replace(".",""))
df_reembolso_resumo["cnpj_cpf_empresa"] = df_reembolso_resumo["cnpj_cpf_empresa"].apply(
lambda x: int(x.replace("-","")))
df_reembolso_resumo["cnpj_cpf_empresa"] = df_reembolso_resumo.cnpj_cpf_empresa.apply(int)

In [13]:
# 1.3. Modelando como grafo
# 1.3.1. Criando tabelas de vínculos
df_vinculos_receitas = df_receitas_resumo.iloc[:,3:]
df_vinculos_receitas.columns = ["nome_empresa","id_empresa","valor","qtde", "nome_deputado"]
df_vinculos_receitas["tipo"] = "doacao"

In [14]:
df_vinculos_reembolsos = df_reembolso_resumo.iloc[:,3:]
df_vinculos_reembolsos.columns = ["nome_empresa", "id_empresa", "valor", "qtde", "nome_deputado"]
df_vinculos_reembolsos = df_vinculos_reembolsos[df_vinculos_reembolsos["nome_deputado"] != "sem_info"].reset_index()
df_vinculos_reembolsos["tipo"] = "reembolso"

In [128]:
df_vinculos_reembolsos

,index,nome_empresa,id_empresa,valor,qtde,nome_deputado,tipo
0,679,A CARABETTI & CIA LTDA,22133011000182,500.37,5,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
1,680,ALMADA & CIA LTDA,16626814000176,703.13,2,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
2,681,ALMADA E CIA LTDA,16626814000257,160.00,3,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
3,682,AMORETTO CAFES EXPRESSO LTDA,8532429000131,2208.80,4,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
4,683,ANDRESSA TAYANE DE ASSIS,21191504000106,8000.00,1,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
5,684,AP MAGALHÃES & CIA LTDA,21027537001006,157.68,1,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
6,685,ARARA AZUL REDE DE POSTOS LTDA,4145589000410,91.57,1,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
7,686,AUTO EIXO 208 NORTE LTDA - ME,7743777000195,100.00,1,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
8,687,AUTO POSTO AEROPORTO LTDA.,8202116000115,350.79,2,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso
9,688,AUTO POSTO AGUIA DE OURO - EIRELI,692806000198,100.00,1,LUIS HENRIQUE DE OLIVEIRA RESENDE,reembolso


In [15]:
df_vinculos_agregados = pd.DataFrame()

In [17]:
df_vinculos_agregados["id_entidade_a"] = list(df_vinculos_receitas["id_empresa"]) + list(df_vinculos_reembolsos["nome_deputado"])
df_vinculos_agregados["id_entidade_b"] = list(df_vinculos_receitas["nome_deputado"]) + list(df_vinculos_reembolsos["id_empresa"])

In [19]:
df_vinculos_agregados["tipo"] = list(df_vinculos_receitas["tipo"]) + list(df_vinculos_reembolsos["tipo"])
df_vinculos_agregados["valor"] = list(df_vinculos_receitas["valor"]) + list(df_vinculos_reembolsos["valor"])
df_vinculos_agregados["qtde"] = list(df_vinculos_receitas["qtde"]) + list(df_vinculos_reembolsos["qtde"])
df_vinculos_agregados["nome_empresa"] = list(df_vinculos_receitas["nome_empresa"]) + list(df_vinculos_reembolsos["nome_empresa"])

In [20]:
df_vinculos_agregados

,id_entidade_a,id_entidade_b,tipo,valor,qtde,nome_empresa
0,20603438000171,FRANCISCO CARLOS OLIVEIRA DE LIMA,doacao,2530.00,3,Comit� Financeiro �nico
1,30842735291,FRANCISCO CARLOS OLIVEIRA DE LIMA,doacao,1500.00,1,FRANCISCO CARLOS OLIVEIRA DE LIMA
2,64575365220,FRANCISCO CARLOS OLIVEIRA DE LIMA,doacao,1500.00,1,PAULO CARLOS O. DA SILVA
3,41271882272,FRANCISCO CARLOS OLIVEIRA DE LIMA,doacao,1500.00,1,VANUSA COSTA LIMA OLIVEIRA
4,20603438000171,JOAQUIM LIRA DE CARVALHO,doacao,530.08,3,Comit� Financeiro �nico
5,20603438000171,JOS AUGUSTO LEANDRO RODRIGUES,doacao,180.00,1,Comit� Financeiro �nico
6,47765933272,JOS AUGUSTO LEANDRO RODRIGUES,doacao,3370.00,3,JOS� AUGUSTO LEANDRO RODRIGUES
7,20603438000171,MARIA SOCORRO SOARES DE OLIVEIRA,doacao,1604.00,3,Comit� Financeiro �nico
8,18874894287,MARIA SOCORRO SOARES DE OLIVEIRA,doacao,650.00,1,MARIA SOCORRO SOARES DE OLIVEIRA
9,06571832291,MARIA SOCORRO SOARES DE OLIVEIRA,doacao,1500.00,1,RAIMUNDA DA SILVA NEGREIROS


In [23]:
G = nx.from_pandas_edgelist(df_vinculos_agregados, "id_entidade_a", 'id_entidade_b', ["tipo","valor","qtde"], create_using=nx.DiGraph())
G.nodes()

NodeView((20603438000171, 'FRANCISCO CARLOS OLIVEIRA DE LIMA', 30842735291, 64575365220, 41271882272, 'JOAQUIM LIRA DE CARVALHO', 'JOS AUGUSTO LEANDRO RODRIGUES', 47765933272, 'MARIA SOCORRO SOARES DE OLIVEIRA', 18874894287, '06571832291', 'MARIA TERESA GAUNA', 'NORMANDO RODRIGUES SALES', 1633510000169, 'SAULO DE FREITAS RIBEIRO', 13813013200, 'SEBASTIO ALVES RODRIGUES MANCHINERY', 65919521287, 75461200206, 30856523291, 18541119491, 'EDSON MAIA NOBRE DE ABREU', 20573849000161, '02370669470', 'FLAVIO HENRIQUE CATO NOGUEIRA', 'JOS FRANCISCO DE LIMA', 16436105468, 'MARCOS ANTNIO VIEIRA DA SILVA', 100727000140, 8842442453, '20863691404', '#NULO', 'PAUDERNEY TOMAZ AVELINO', 1666158000168, 22797468000191, 5748909200, 83696628204, 156701200, 33482110263, 43637949253, 5761360187, 4782407000179, 4565289000147, 89979800259, 10238315000125, 33462127268, 4928297000100, 4307088000140, 2887227000126, 5516984000181, 20563951000186, 9257826000105, 14631881291, 38447428249, 37185411220, 5445663000133, 